파이토치의 모델 학습은?

경사하강법 -> cost function 이 감소하는 방향 그렇다면 미분을 계속 수행해야하는데 파이토치의 자동미분을 사용하여 미분 계산을 자동화한다.



In [28]:
w = torch.tensor(2.0, requires_grad=True) # requires_grad 변수값을 변화시킬 수 있는 부분
y = w**2
z = 2*y + 5

`.backward()` : 수식의 기울기 계산

In [29]:
z.backward()

`.grad` 를 이요하여 미분한 값 확인

In [30]:
w.grad

tensor(8.)

# 로지스틱 회귀

In [1]:
## 로지스틱 회귀 구현하기

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [16]:
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2,  3], [3, 1], [4, 3], [5, 3], [6, 2]] # 6,2
y_data = [[0], [0], [0], [1], [1], [1]] # 6, 1

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)


In [4]:
# 가중치와 bias

# x_data 2개의 값을 지니므로 ==> 가중치도 2개
W = torch.zeros((2, 1), requires_grad=True) # requires_grad : 변수는 학습을 통해 값이 계속 변경되는 변수
b = torch.zeros(1, requires_grad=True)


In [5]:
# z = Wx + b
# H = sigmoid(z)

z = x_train.matmul(W) + b
hypothesis = 1 / (1 + torch.exp(-z))



In [6]:
print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


가설을 보면 모두 0.5 값의 예측값 벡터가 나오게 된다... \
--> 안좋은 가중치임! 변경이 필요!

In [9]:
# cost function - 직접구현하기

losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
cost = losses.mean()
cost

tensor(0.6931, grad_fn=<MeanBackward0>)

In [10]:
# functional 함수 이용하기

F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

In [11]:
# 모델링

# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad() # 파이토치 미분의 기울기를 누적시키는 특징 있기때문에 늘 초기화
    cost.backward() # 미분
    optimizer.step() # 업데이트

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [12]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis) # 확률벡터

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [14]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print([1 if i else 0 for i in prediction ])

[0, 0, 0, 1, 1, 1]


In [15]:
print(W, b)

tensor([[3.2530],
        [1.5179]], requires_grad=True) tensor([-14.4819], requires_grad=True)


# nn.Module 구현하는 logistic regression

In [17]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)


In [18]:
model = nn.Sequential(
    nn.Linear(2, 1),
    nn.Sigmoid()
)
# 모델 컴파일 과정이 없다.

In [19]:
model(x_train)

tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward0>)

In [33]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

In [34]:
# 학습 후 W, b
list(model.parameters())

[Parameter containing:
 tensor([[3.2534, 1.5181]], requires_grad=True),
 Parameter containing:
 tensor([-14.4839], requires_grad=True)]

In [35]:
model(x_train)

tensor([[2.7616e-04],
        [3.1595e-02],
        [3.8959e-02],
        [9.5624e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)

In [37]:
[1 if i >0.5 else 0 for i in model(x_train)]
# pred = hypothesis >= torch.FloatTensor([0.5])
# [1 if i else 0 for i in pred]

[0, 0, 0, 1, 1, 1]